In [45]:
from nltk.corpus import gutenberg
macbeth = gutenberg.raw('shakespeare-macbeth.txt')
#text = macbeth.replace('\n', '')
import sys
from collections import defaultdict

class Importance_calculations:

    def __init__(self, file):
        self.data = self.filter(file)

    def filter(self, str):
        str = str.lower()
        i = [x for x in range(97)] 
        j = [x for x in range(123,256)]
        k = i + j
        for c in [chr(i) for i in k]:
            str = str.replace(c, '')
        return str

    def importance_dictionary(self):
        dictionary = []
        counter = defaultdict(int)
        phrase = ""
        for char in self.data:
            new_phrase = phrase + char
            
            if new_phrase in dictionary:
                phrase = new_phrase
            else:
                counter[phrase] += 1
                dictionary.append(new_phrase)
                # phrase = char
                phrase = ''
        print('importance dictionary:', dictionary)
        return counter, dictionary

    def rank(self):
        counter, dictionary = self.importance_dictionary()
        dictionary.sort(key=lambda str: len(str)*counter[str], reverse=True)
        return dictionary, counter
    
    def importance_sorting(self):
        ranked_dict, counter = self.rank()
        print(counter)
        return ranked_dict, counter
    
    def encode_rank(self):
        ranked_dict, _ = self.rank()
        ranking_levels = {}
        rank = 1
        for e in ranked_dict:
            ranking_levels[e] = rank
            rank += 1
        print('ranking_levels:', ranking_levels)
        return ranking_levels


file = open('no_rel/data.txt', 'r')
# text = file.readlines()[0]
text = "aababbabbaababba"
Test = Importance_calculations(text)
rectified = Test.filter(text)
z,occ = Test.importance_sorting()
# print(z)
# print(occ)


u = Test.encode_rank()
print(sys.getsizeof(rectified))

importance dictionary: ['a', 'ab', 'abb', 'abba', 'aba', 'b', 'ba']
defaultdict(<class 'int'>, {'': 2, 'a': 1, 'ab': 2, 'abb': 1, 'b': 1, 'abba': 0, 'aba': 0, 'ba': 0})
importance dictionary: ['a', 'ab', 'abb', 'abba', 'aba', 'b', 'ba']
ranking_levels: {'ab': 1, 'abb': 2, 'a': 3, 'b': 4, 'abba': 5, 'aba': 6, 'ba': 7}
65


In [64]:
encode_order = {'ab': 1, 'abb': 2, 'a': 3, 'b': 4, 'abba': 5, 'aba': 6, 'ba': 7}
#encode_order = u
text = "aababbabbaababba"
#text = rectified

class Encoding:

    def __init__(self, phrase, rankings_encode):
        self.data = phrase 
        self.encode_order = rankings_encode
        print(rankings_encode)

    def encoder(self):
        
        accum = ''
        encoded = ''
        appeared_before = []
        code_with_space = []
        for s in self.data:
            #print(accum)
            pattern = accum+s
            if pattern in appeared_before:
                accum = pattern 
            else:
                appeared_before.append(pattern)
                # print('appeared', appeared_before)
                if len(pattern) == 1:
                    encoded += '# '+pattern+' '
                    code_with_space.append("#"+pattern) 
                else:
                    encoded = encoded + str(self.encode_order[accum]) +' '+s+' '
                    code_with_space.append(str(self.encode_order[accum])+s)
                accum = ''
        #print(address_code)
        return encoded, appeared_before

    def encoded(self):
        code,_ = self.encoder()
        return code.replace(' ', '')

    def ambiguity_solver(self,):
        code_with_space,list_code = self.encoder()
        accumulator = code_with_space.split()
        running_dictionary = {}
        unambiguous = ''

        def return_key(dictionary, idx):
            return list(dictionary.keys())[list(dictionary.values()).index(idx)]

        for i in range(len(list_code)-1):
            s = accumulator[2*i], accumulator[2*i+1] 
            if s[0] != '#' :
                if (accumulator[2*i+2] != '#') and (int(accumulator[2*i+2]) not in list(running_dictionary.values())):
                    unambiguous += s.join('')
                    prev = return_key(running_dictionary, int(s[0]))
                    running_dictionary[prev + s[1]] = int(accumulator[2*i+2])
                elif (accumulator[2*i+2] != '#') and int(accumulator[2*i+2]) in list(running_dictionary.values()):
                    if return_key(running_dictionary,int(accumulator[2*i+2]))+s[1] != return_key(running_dictionary, int(s[0]))+s[1]:
                        actual_rank = self.encode_order[return_key(running_dictionary, int(s[0])) + s[1]]
                        running_dictionary[return_key(running_dictionary, int(s[0])) + s[1]] = actual_rank
                        unambiguous = unambiguous + s.join('') + '(' + actual_rank + ')'
                    else:
                        unambiguous += s.join('')
                else:
                    prev = return_key(running_dictionary, int(s[0]))
                    actual_rank = self.encode_order[prev + s[1]]
                    running_dictionary[prev + s[1]] = actual_rank
                    unambiguous = unambiguous + s.join('') + '(' + actual_rank + ')'
            if s[0] == '#':
                if accumulator[2*i+2] == '#':
                    actual_rank = self.encode_order[s[1]]
                    running_dictionary[s[1]] = actual_rank
                    unambiguous = unambiguous + s.join('') + '(' + actual_rank + ')'
                elif accumulator[2*i+2] != '#' and return_key(running_dictionary, int(accumulator[2*i+2])) != s[1]:
                    actual_rank = self.encode_order[s[1]]
                    running_dictionary[s[1]] = actual_rank
                    unambiguous = unambiguous + s.join('') + '(' + actual_rank + ')'
                else:
                    actual_rank = int(accumulator[2*i+2])
                    running_dictionary[s[1]] = actual_rank 
                    unambiguous = unambiguous + s.join('')

        return unambiguous



class Decoding:

    def __init__(self, code, rankings_encode):
        self.code = code 
        self.priority = rankings_encode

    def decoder(self):
        pass

test = Encoding(text, encode_order)
code = test.encoded()

print(sys.getsizeof(code))
print(code)
print(test.ambiguity_solver())
#test.ambiguities()

{'ab': 1, 'abb': 2, 'a': 3, 'b': 4, 'abba': 5, 'aba': 6, 'ba': 7}
63
#a3b1b2a1a#b4a


ValueError: 3 is not in list

In [60]:
encode_order = {'ab': 1, 'abb': 2, 'a': 3, 'b': 4, 'abba': 5, 'aba': 6, 'ba': 7}
arr = []

arr.append(encode_order[''])
arr

KeyError: ''